# Using hugging face to parse a website and summarize

In [ ]:
!pip -q install transformers

In [ ]:
!pip -q install BeautifulSoup4

In [ ]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

## Load summarisation pipeline

In [ ]:
summarizer = pipeline("summarization")

### Get blog post

In [ ]:
URL = "https://medium.com/the-ascent/i-discovered-its-never-too-late-to-invest-in-bitcoin-if-you-understand-it-d0848141144b"

In [ ]:
r = requests.get(URL)

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [ ]:
ARTICLE

### Chunk text

In [ ]:
max_chunk = 500

In [ ]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [ ]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

In [ ]:
len(chunks)

### Summarize text

In [ ]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [ ]:
res[0]

In [ ]:
' '.join([summ['summary_text'] for summ in res])

In [ ]:
text = ' '.join([summ['summary_text'] for summ in res])

### Output to file

In [ ]:
with open('summary.txt', 'w') as f:
    f.write(text)